In [2]:
# Function to process and display images one at a time
def process_and_display_images(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        # Construct full file path
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # Check if it is a file and has an image extension
        if os.path.isfile(input_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            # Read the image
            image = cv2.imread(input_path)
            if image is not None:
                # Enhance the image
                enhanced_image = enhance_image(image)
                
                # Display the original image
                cv2.imshow("Original Image", image)
                # Display the enhanced image
                cv2.imshow("Enhanced Image", enhanced_image)
                
                # Wait for a key press to move on
                print(f"Displaying: {filename}")
                key = cv2.waitKey(0)  # Wait indefinitely until a key is pressed
                if key == 27:  # If 'Esc' is pressed, exit the loop
                    print("Exiting...")
                    break
                
                # Close the display windows
                cv2.destroyAllWindows()
                
                # Save the enhanced image to the output folder
                cv2.imwrite(output_path, enhanced_image)
                print(f"Processed and saved: {output_path}")
            else:
                print(f"Failed to read image: {input_path}")

In [ ]:
def halve_image_dimensions(image):
    """
    Halves the dimensions of an image.
    Parameters:
    - image: Input image (NumPy array).
    Returns:
    - Resized image.
    """
    # Get current dimensions
    height, width = image.shape[:2]
    print(f"Original size: {width}x{height}")

    # Halve the dimensions
    new_width, new_height = width // 2, height // 2

    # Resize the image
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
    print(f"Resized image to: {new_width}x{new_height}")
    return resized_image

In [ ]:
net = cv2.dnn.readNetFromCaffe(prototxt, caffemodel)
def detect_swimmer_with_mobilenet(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image {image_path}")
        return None

    image = enhance_image(image)
    
    # Prepare the image for MobileNet SSD
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    h, w = image.shape[:2]
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.2:  # Confidence threshold 
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype("int")
            x_center = int((x1 + x2) / 2)
            y_center = int((y1 + y2) / 2)
            return np.array([[x_center, y_center]]), image  # Return center point and image
    return None, image  # Return None if no detection, along with the image


In [ ]:
def detect_swimmer_with_mobilenet_1(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image {image_path}")
        return None

    image = enhance_image(image)
    
    # Get the original dimensions of the image
    h, w = image.shape[:2]
    
    # Prepare the image for MobileNet SSD
    blob = cv2.dnn.blobFromImage(image, 0.007843, (w, h), 127.5)
    net.setInput(blob)
    detections = net.forward()

    h, w = image.shape[:2]
    grid_color = (255, 255, 255)  # White color for the grid

    # Draw the grid
    num_rows, num_cols = 10, 10  # Define grid dimensions
    for row in range(1, num_rows):
        y = row * h // num_rows
        cv2.line(image, (0, y), (w, y), grid_color, 1)
    for col in range(1, num_cols):
        x = col * w // num_cols
        cv2.line(image, (x, 0), (x, h), grid_color, 1)

    detected_points = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Increased confidence threshold
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype("int")
            x_center = int((x1 + x2) / 2)
            y_center = int((y1 + y2) / 2)

            # Debug bounding box and center
            print(f"Bounding box: ({x1}, {y1}), ({x2}, {y2})")
            print(f"Center: ({x_center}, {y_center}), Confidence: {confidence}")

            # Draw bounding box
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
            
            # Draw the red point for the center
            cv2.circle(image, (x_center, y_center), 5, (0, 0, 255), -1)
            detected_points.append([x_center, y_center])

            # Add additional points around the center
            offsets = [(-10, 0), (10, 0), (0, -10), (0, 10), (-10, -10)]
            for dx, dy in offsets:
                new_x, new_y = x_center + dx, y_center + dy
                cv2.circle(image, (new_x, new_y), 5, (255, 0, 0), -1)  # Blue points
                detected_points.append([new_x, new_y])

    if not detected_points:
        print("No swimmer detected.")
        return None, image

    return np.array(detected_points), image

In [1]:
net = cv2.dnn.readNetFromCaffe(prototxt, caffemodel)
def detect_swimmer_with_mobilenet(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image {image_path}")
        return None

    image = enhance_image(image)

    # Get the original dimensions of the image
    h, w = image.shape[:2]
    
    # Prepare the image for MobileNet SSD
    blob = cv2.dnn.blobFromImage(image, 0.007843, (w, h), 127.5)
    net.setInput(blob)
    detections = net.forward()

    h, w = image.shape[:2]
    grid_color = (255, 255, 255)  # White color for the grid

    # Draw the grid
    num_rows, num_cols = 10, 10  # Define grid dimensions
    for row in range(1, num_rows):
        y = row * h // num_rows
        cv2.line(image, (0, y), (w, y), grid_color, 1)
    for col in range(1, num_cols):
        x = col * w // num_cols
        cv2.line(image, (x, 0), (x, h), grid_color, 1)

    detected_points = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.1:  # Confidence threshold
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype("int")
            x_center = int((x1 + x2) / 2)
            y_center = int((y1 + y2) / 2)

            # Draw the red point for the center
            cv2.circle(image, (x_center, y_center), 5, (0, 0, 255), -1)
            detected_points.append([x_center, y_center])

            # Add additional points around the center
            offsets = [(-10, 0), (10, 0), (0, -10), (0, 10), (-10, -10)]
            for dx, dy in offsets:
                new_x, new_y = x_center + dx, y_center + dy
                cv2.circle(image, (new_x, new_y), 5, (255, 0, 0), -1)  # Blue points
                detected_points.append([new_x, new_y])

    if not detected_points:
        print("No swimmer detected.")
        return None, image

    return np.array(detected_points), image

NameError: name 'cv2' is not defined